<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/C%C3%B3digos%20convencionales/Turbo_c%C3%B3digos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para la ejecución de estos códigos es necesario incluir las librerias de la biblioteca pyturbo en Google Drive, esta biblioteca se encuentra en: https://github.com/DaulPavid/pyturbo

In [ ]:
#@title Turbo códigos

import sys
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
from turbo import TurboEncoder, AWGN, TurboDecoder

def canal_bsc(mensaje, ruido):
    msg_bits = ((mensaje + 1)/2).astype(int)
    msg_canal_bits = np.bitwise_xor(msg_bits, ruido)
    msg_canal = (msg_canal_bits * 2) - 1
    return msg_canal

def calculate_ber(original_bits, decoded_bits):
    error_count = sum([x != y for x, y in zip(original_bits, decoded_bits)])
    ber = error_count / len(original_bits)
    return ber
def guardar_matriz_en_csv(matriz, nombre_archivo):
    # Convertir la matriz a un DataFrame de pandas
    df = pd.DataFrame(matriz.T)  # Transponer para que cada columna corresponda a una probabilidad

    # Asignar nombres a las columnas como Probabilidad 1, Probabilidad 2, etc.
    df.columns = [f'Probabilidad {i + 1}' for i in range(matriz.shape[0])]

    # Guardar el DataFrame en un archivo CSV
    df.to_csv(nombre_archivo, index=False)
def turbo_encode(message_bits):
    block_size = len(message_bits)
    interleaver = random.sample(range(0, block_size), block_size)
    encoder = TurboEncoder(interleaver)
    encoded_vector = encoder.execute(message_bits)
    return encoded_vector, interleaver

def turbo_decode(received_vector, interleaver):
    decoder = TurboDecoder(interleaver)
    decoded_vector = decoder.execute(received_vector)
    decoded_vector = [int(b > 0.0) for b in decoded_vector]
    return decoded_vector

def tanda_train(batch_size, block_len, code_rate, qs, _, progreso_tanda):
    ber_q = np.zeros((len(qs), batch_size))
    for j, q in enumerate(qs):
        data = np.random.randint(0, 2, (batch_size, block_len, 1))
        noise = np.random.binomial(1, q, size=(batch_size, (block_len*code_rate) + 6, 1)).astype(int)
        for i in range(0, batch_size):
            progreso_tanda.update(1)
            raw_data = (data[i:i+1, :, :]).squeeze().squeeze()
            batch_noise = (noise[i:i+1, :, :]).squeeze().squeeze()
            encoded_vector, interleaver = turbo_encode(raw_data)
            coded_antipodal = (encoded_vector * 2) - 1
            channel_vector = canal_bsc(coded_antipodal, batch_noise)
            decoded_vector = np.array(turbo_decode(channel_vector, interleaver))
            received_data = decoded_vector[:len(raw_data)]
            ber = calculate_ber(raw_data, received_data)
            ber_q[j][i] = ber
    return ber_q

def guardar_csv(data, filename):
    filepath = filename  # Ajusta la ruta según dónde quieras guardar el archivo
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Q values', 'Average BER'])
        for row in data:
            writer.writerow(row)



if __name__ == "__main__":
    names = ["turbo_1_3.csv"]

    for name in names:
        block_len = 50
        batch_size = 100
        code_rate = 3
        num_block = 100000
        num_test_batch = int(num_block/batch_size)
        qs = np.linspace(0.0001, 0.15, 20)
        num_epoch_test = 100

        arrays_q_turbo = []
        print(f"Total:", num_epoch_test, "tandas")
        progreso_tanda = tqdm(total=num_epoch_test*20*batch_size, desc=f"Progreso GENERAL")
        for _ in range(num_epoch_test):
            ber_q_turbo = tanda_train(batch_size, block_len, code_rate, qs, _, progreso_tanda)
            arrays_q_turbo.append(ber_q_turbo)
            #progreso_tanda.update(1)

        qs_reshaped = qs.reshape(20, 1)

        turbo_stack = np.hstack(arrays_q_turbo)
        turbo_data = np.hstack((qs_reshaped, turbo_stack))
        guardar_matriz_en_csv(turbo_data, "turbo_1_3.csv")
        print(turbo_data.shape)